# Word2Vec: Obtain word embeddings

## 0. Introduction

**word2vec**は単語の分散表現を生成するツールで、Mikolov et al[1]によって提案されました。単語の意味が近いほど類似度が大きくなるように、word2vecは各単語に実ベクトルを割り当てます。

ここで、**分散表現**とは各単語に対して実ベクトルを割り当て、そのベクトルで単語を表現することです。分散表現で単語を表現する場合、そのベクトルを**word embeddings(単語埋め込み)** と呼びます。このNotebookでは、Penn Tree Bankのデータセットからword embeddingsを獲得する方法を説明します。

さて、そもそも単語の意味とはなんでしょうか。人であれば「動物」と「犬」という単語が似ているというのはなんとなく分かります。しかし、word2vecは何の情報を元に、「動物」と「犬」は似ているとか、「食べ物」と「犬」は似ていないといった意味の類似度を学習すれば良いのでしょうか。

## 1. 基本的なアイデア

word2vecは単語の意味の類似度を単純な情報から学習します。それは文章における単語の並びです、つまりある単語の意味は、その単語の周囲の単語で決まるというアイデアです。 このアイデアは**distributional hypothesis(分布仮設)**[2]に基づいています。

学習対象の単語を**center word**、その周囲の単語を**context words**と呼びます。ウィンドウサイズ`C`に応じてcontex wordの数は変わります。

例として、**The cute cat jumps over the lazy dog.**という文で説明を行います。
以下の図は全てcenter wordをcatとした場合のものです。
ウィンドウサイズ`C`に応じて、catを学習する際に使用するcontex wordが変わることがわかると思います。

![center_context_word.png](https://docs.chainer.org/en/stable/_images/center_context_word.png)

## 2. 主なアルゴリズム

word2vecと呼ばれる手法は実は**Skip-gram**と**CBoW**という2つの手法の総称です。

To explain the models with the figures below, we will use the following symbols.

* $|\mathcal{V}|$ : ボキャブラリ数
* $D$              :  埋め込みベクトルのサイズ
* ${\bf v}_t$     : center wordのone-hotベクトル
* $V_{\pm C}$     : ${\bf v}_t$の周囲のcontext wordのone-hotベクトルの集合、つまり$\{{\bf v}_{t+c}\}_{c=-C}^C \backslash {\bf v}_t$
* ${\bf l}_H$     : 入力単語に対する埋め込みベクトル
* ${\bf l}_O$     : ネットワークの出力ベクトル
* ${\bf W}_H$     : 入力に対する埋め込み行列
* ${\bf W}_O$     : 出力に対する埋め込み行列

**Note**

**negative sampling**や**hierarchical softmax**をロス関数に使うことが一般的だが、**すべての単語に対するsoftmax関数**を使い、説明を簡略化するため他の説明は省略します。

### 2.1 Skip-gram

このモデルは、 center wordが与えられたときにその周囲のcontext words $V_{t \pm C}$を予測するように学習します。この時、入力に対する埋め込み行列$W_H$の各行が各単語の分散表現になります。

center word ${\bf v}_t$をネットワークに入力したとき、以下のようにしてcontext words $\hat{\bf v}_{t+i} \in V_{t \pm C}$を予測することができます

1. 入力されたcenter wordに対する埋め込みベクトルを計算する: ${\bf l}_H = {\bf W}_H {\bf v}_t$
2. 埋め込みベクトルを使って出力ベクトルを計算する: ${\bf l}_O = {\bf W}_O {\bf l}_H$
3. context wordの確率ベクトルを計算する: $\hat{\bf v}_{t+i} = \text{softmax}({\bf l}_O)$

$|\mathcal{V}|$次元のベクトル$\hat{\bf v}_{t+i}$の各要素は、各単語がcontext wordである確率です。そのため、確率$p({\bf v}_{t+i} \mid {\bf v}_t)$は、context wordのone-hotベクトル${\bf v}_{t+i}$と確率ベクトル$\hat{\bf v}_{t+i}$の内積で計算することができます。

\begin{eqnarray}
p({\bf v}_{t+i} \mid {\bf v}_t) = {\bf v}_{t+i}^T \hat{\bf v}_{t+i}
\end{eqnarray}

そして、center word ${\bf v}_t$に対するすべてのcontext word$V_{t \pm C}$のロス関数は以下で計算することができます。


\begin{eqnarray}
L(V_{t \pm C} | {\bf v}_t; {\bf W}_H, {\bf W}_O)
&=& \sum_{V_{t \pm C}} -\log\left(p({\bf v}_{t+i} \mid {\bf v}_t)\right) \\
&=& \sum_{V_{t \pm C}} -\log({\bf v}_{t+i}^T \hat{\bf v}_{t+i})
\end{eqnarray}

### 2.2 Continuous Bag of Words (CBoW)

このモデルは、context word $V_{t \pm C}$ が与えられたときにcenter word ${\bf v}_t$を予測するように学習します。

context words $V_{t \pm C}$をネットワークに与えたとき、以下のようにcenter word $\hat{v}_t$の確率を計算することができます。

1. すべてのcontext wordに対する埋め込みベクトルの平均を計算します: ${\bf l}_H = \frac{1}{2C} \sum_{V_{t \pm C}} {\bf W}_H {\bf v}_{t+i}$
2. 埋め込みベクトルを使って出力ベクトルを計算します: ${\bf l}_O = {\bf W}_O {\bf l}_H$
3. center wordの確率ベクトルを計算する: $\hat{\bf v}_t = \text{softmax}({\bf l}_O)$

$|\mathcal{V}|$次元のベクトル$\hat{\bf v}_t$の各要素は、各単語がcenter wordである確率です。そのため、確率$p({\bf v}_t \mid V_{t \pm C})$は、center wordのone-hotベクトル${\bf v}_{t}$と確率ベクトル$\hat{\bf v}_{t}$の内積で計算することができます。

\begin{eqnarray}
p({\bf v}_{t} \mid V_{t \pm C}) = {\bf v}_{t}^T \hat{\bf v}_{t}
\end{eqnarray}

The loss function for the center word prediction is defined as follows:

そして、context word$V_{t \pm C}$対するcenter word ${\bf v}_t$のロス関数は以下で計算することができます。


\begin{eqnarray}
L({\bf v}_t|V_{t \pm C}; W_H, W_O)
&=& -\log(p({\bf v}_t|V_{t \pm C})) \\
&=& -\log({\bf v}_t^T \hat{\bf v}_t)
\end{eqnarray}

## 3. Skip-gramの詳細

本チュートリアルでは、以下の観点からSkip-gramをメインで扱います。

1. 学習アルゴリズムがCBoWに比べて理解しやすい
2. 単語数が増えても精度が落ちにくく、スケールしやすい

skip-gramのアルゴリズムを理解するために、以下の設定で具体的な例から考えてみましょう:

* ボキャブラリ数 $|\mathcal{V}|$ は10。
* 埋め込みベクトルのサイズ$D$は2。
* Center wordはdog。
* Context wordはanimal。

そして、以下の工程をcontext word数回繰り返します。

1. dogのone-hotベクトルは`[0 0 1 0 0 0 0 0 0 0]`で、 これをモデルに入力する。
2. このとき、埋め込み行列${\bf W}_H$の３番目の行${\bf l}_H$がdogの埋め込みベクトルとなる。
3. そして、出力ベクトル${\bf l}_O$を計算するため、${\bf W}_O$と${\bf l}_H$の積を計算する。
4. $c$の位置にあるcontext wordの確率ベクトル$\hat{\bf v}_{t+c}$を予測するため${\bf l}_O$をsoftmax関数に入力する。
5. $\hat{\bf v}_{t+c}$と animalのone-hotベクトル`[1 0 0 0 0 0 0 0 0 0 0]`の誤差を計算する。
6. 誤差を伝播させてネットワークのパラメータを更新する。

![skipgram_detail.png](https://docs.chainer.org/en/stable/_images/skipgram_detail.png)

## 4. Chainerによるskip-gram実装方法

GitHubレポジトリ上のexamples内にword2vecに関するコードがあるので、それに基づいて説明をしていきます。[chainer/examples/word2vec](https://github.com/chainer/chainer/tree/master/examples/word2vec)

まずは、以下のセルを実行して、ChainerとそのGPUバックエンドであるCuPyをインストールします。Colaboratoryの「ランタイムのタイプ」がGPUであれば、GPUをバックエンドとしてChainerを動かすことができます。

In [0]:
!curl https://colab.chainer.org/install | sh -

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcusparse8.0 is already the newest version (8.0.61-1).
libnvrtc8.0 is already the newest version (8.0.61-1).
libnvtoolsext1 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


### 4.1 準備

必要なパッケージを`import`しましょう。

In [0]:
import argparse
import collections

import numpy as np
import six

import chainer
from chainer import cuda
import chainer.functions as F
import chainer.initializers as I
import chainer.links as L
import chainer.optimizers as O
from chainer import reporter
from chainer import training
from chainer.training import extensions

### 4.2 skip-gramモデルの定義

次にskip-gramのネットワーク構造を定義しましょう。

In [0]:
class SkipGram(chainer.Chain):

    def __init__(self, n_vocab, n_units):
        super().__init__()
        with self.init_scope():
            self.embed = L.EmbedID(
                n_vocab, n_units, initialW=I.Uniform(1. / n_units))
            self.out = L.Linear(n_units, n_vocab, initialW=0)

    def __call__(self, x, context):
        e = self.embed(context)
        shape = e.shape
        x = F.broadcast_to(x[:, None], (shape[0], shape[1]))
        e = F.reshape(e, (shape[0] * shape[1], shape[2]))
        x = F.reshape(x, (shape[0] * shape[1],))
        center_predictions = self.out(e)
        loss = F.softmax_cross_entropy(center_predictions, x)
        reporter.report({'loss': loss}, self)
        return loss

**Note**

- 重み行列`self.embed.W`は入力`x`に対する埋め込み行列です。
- `__call__`は center wordの単語ID `x`とcontext wordの単語ID `contexts`を入力として取ります。そして、ロス関数`softmax_cross_entropy`で計算された誤差を出力します。
- 注意してもらいたいのが、 `x`と`contexts`の形がそれぞれ`(batch_size,)`と`(batch_size, n_context)`になっていることです。
- `batch_size`はミニバッチサイズを意味し、 `n_context`はcontext word数を意味します。

まず、`e = self.embed(contexts)`で`contexts`に対応する分散表現を取得しています。

そして、 `F.broadcast_to(x[:, None], (shape[0], shape[1]))`とすることで、`x`(`(batch_size,)`) を`(batch_size, n_context)`の形にブロードキャストします。このとき、 列方向に`n_context`回だけ同じ値がコピーされます。そして、ブロードキャストされた`x`は１次元ベクトルにreshapeされ、`(batchsize * n_context,)`になります。一方で、`e`は`(batch_size * n_context, n_units)`の形にreshapeされます。

注意してもらいたいのが、skip-gramの場合、center wordとcontext wordは1対1で対応するため、center wordとcontext wordを入れ替えてモデル化しても問題がないです。そのため、上記ではcenter wordとcontext wordを入れ替えて学習させているように見えますが、問題はありません。なぜこのようなことをするかと言うと、CBoWモデルとコードの整合性が取りやすいからです。

### 4.3 datasetとiteratorの準備

Chainer'が用意するユーティリティ関数`get_ptb_words()`を使って、Penn Tree Bank (PTB)のデータセットをダウンロードしましょう。

In [0]:
train, val, _ = chainer.datasets.get_ptb_words()
n_vocab = max(train) + 1  # The minimum word ID is 0

center wordと、そのcontext wordを含むミニバッチを生成するIteratorを定義しましょう。

In [0]:
class WindowIterator(chainer.dataset.Iterator):

    def __init__(self, dataset, window, batch_size, repeat=True):
        self.dataset = np.array(dataset, np.int32)
        self.window = window
        self.batch_size = batch_size
        self._repeat = repeat

        self.order = np.random.permutation(
            len(dataset) - window * 2).astype(np.int32)
        self.order += window
        self.current_position = 0
        self.epoch = 0
        self.is_new_epoch = False

    def __next__(self):
        if not self._repeat and self.epoch > 0:
            raise StopIteration

        i = self.current_position
        i_end = i + self.batch_size
        position = self.order[i: i_end]
        w = np.random.randint(self.window - 1) + 1
        offset = np.concatenate([np.arange(-w, 0), np.arange(1, w + 1)])
        pos = position[:, None] + offset[None, :]
        context = self.dataset.take(pos)
        center = self.dataset.take(position)

        if i_end >= len(self.order):
            np.random.shuffle(self.order)
            self.epoch += 1
            self.is_new_epoch = True
            self.current_position = 0
        else:
            self.is_new_epoch = False
            self.current_position = i_end

        return center, context

    @property
    def epoch_detail(self):
        return self.epoch + float(self.current_position) / len(self.order)

    def serialize(self, serializer):
        self.current_position = serializer('current_position',
                                           self.current_position)
        self.epoch = serializer('epoch', self.epoch)
        self.is_new_epoch = serializer('is_new_epoch', self.is_new_epoch)
        if self._order is not None:
            serializer('_order', self._order)

def convert(batch, device):
    center, context = batch
    if device >= 0:
        center = cuda.to_gpu(center)
        context = cuda.to_gpu(context)
    return center, context

- コンストラクタの中で、文書中の単語の位置をシャッフルしたリスト`self.order`を作成しています。文書からランダムに単語を選択し学習するようにするためです。ウィンドウサイズ分だけ最初と最後を切り取った単語の位置がシャッフルされて入っています。
- イテレータの定義`__next__`は、コンストラクタのパラメータに従ってミニバッチサイズ個のcenter word `center`とcontext word `context`を返します。
- `self.order[i:i_end]`で、単語の位置をシャッフルしたリスト`self.order`から`batch_size`分のcenter wordのインデックス`position`を生成します。(`position`は後で`self.dataset.take`によってcenter word `center`に変換されます。)
- `np.concatenate([np.arange(-w, 0), np.arange(1, w + 1)])`で、ウインドウを表現するオフセット`offset`を作成しています。
- `position[:, None] + offset[None, :]`によって、それぞれのcenter wordに対するcontext word のインデックス`pos`を生成します。`pos`は後で`self.dataset.take`によってcontext word  `context`に変換されます。

### 4.4 model, optimizer, updaterの準備

In [0]:
unit = 100  # number of hidden units
window = 5
batchsize = 1000
gpu = 0

# Instantiate model
model = SkipGram(n_vocab, unit)

if gpu >= 0:
    model.to_gpu(gpu)

# Create optimizer
optimizer = O.Adam()
optimizer.setup(model)

# Create iterators for both train and val datasets
train_iter = WindowIterator(train, window, batchsize)
val_iter = WindowIterator(val, window, batchsize, repeat=False)

# Create updater
updater = training.StandardUpdater(
    train_iter, optimizer, converter=convert, device=gpu)

### 4.5 trainingの開始

In [0]:
epoch = 100

trainer = training.Trainer(updater, (epoch, 'epoch'), out='word2vec_result')
trainer.extend(extensions.Evaluator(val_iter, model, converter=convert, device=gpu))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'elapsed_time']))
trainer.run()

epoch       main/loss   validation/main/loss  elapsed_time
1           6.87314     6.48688               54.154        
2           6.44018     6.40645               107.352       
3           6.35021     6.3558                159.544       
4           6.28615     6.31679               212.612       
5           6.23762     6.28779               266.059       
6           6.19942     6.22658               319.874       
7           6.15986     6.20715               372.798       
8           6.13787     6.21461               426.456       
9           6.10637     6.24927               479.725       
10          6.08759     6.23192               532.966       
11          6.06768     6.19332               586.339       
12          6.04607     6.17291               639.295       
13          6.0321      6.21226               692.67        
14          6.02178     6.18489               746.599       
15          6.00098     6.17341               799.408       
16          5.99099     6.

In [0]:
vocab = chainer.datasets.get_ptb_words_vocabulary()
index2word = {wid: word for word, wid in six.iteritems(vocab)}

# Save the word2vec model
with open('word2vec.model', 'w') as f:
    f.write('%d %d\n' % (len(index2word), unit))
    w = cuda.to_cpu(model.embed.W.data)
    for i, wi in enumerate(w):
        v = ' '.join(map(str, wi))
        f.write('%s %s\n' % (index2word[i], v))

### 4.6 似た単語の検索

In [0]:
import numpy
import six

n_result = 5  # number of search result to show


with open('word2vec.model', 'r') as f:
    ss = f.readline().split()
    n_vocab, n_units = int(ss[0]), int(ss[1])
    word2index = {}
    index2word = {}
    w = numpy.empty((n_vocab, n_units), dtype=numpy.float32)
    for i, line in enumerate(f):
        ss = line.split()
        assert len(ss) == n_units + 1
        word = ss[0]
        word2index[word] = i
        index2word[i] = word
        w[i] = numpy.array([float(s) for s in ss[1:]], dtype=numpy.float32)


s = numpy.sqrt((w * w).sum(1))
w /= s.reshape((s.shape[0], 1))  # normalize

In [0]:
def search(query):
  if query not in word2index:
    print('"{0}" is not found'.format(query))
    return

  v = w[word2index[query]]
  similarity = w.dot(v)
  print('query: {}'.format(query))

  count = 0
  for i in (-similarity).argsort():
      if numpy.isnan(similarity[i]):
          continue
      if index2word[i] == query:
          continue
      print('{0}: {1}'.format(index2word[i], similarity[i]))
      count += 1
      if count == n_result:
          return

appleで検索してみましょう。

In [0]:
query = "apple"
search(query)

query: apple
computer: 0.5457335710525513
compaq: 0.5068206191062927
microsoft: 0.4654524028301239
network: 0.42985647916793823
trotter: 0.42716777324676514


## 5. Reference

* [1] [Mikolov, Tomas; et al. “Efficient Estimation of Word Representations in Vector Space”. arXiv:1301.3781](https://arxiv.org/abs/1301.3781)
* [2] [Distributional Hypothesis](https://aclweb.org/aclwiki/Distributional_Hypothesis)
